In [ ]:
! pip install stop-words
!pip install pyLDAvis

In [35]:
import re
import numpy as np
import pandas as pd

from pprint import pprint
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel

In [3]:
# spacy for lemmatization
import spacy

In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [19]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [24]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [25]:
# Import Dataset
df = pd.read_table("Politics.txt",header=None)
df.head()

,0
0,x
1,"politics (from greek: πολιτικά, politiká, 'af..."
2,chinese philosopher confucius also wrote poli...
3,"in contrast with voluntary theories, these arg..."
4,"according to david easton, ""a political system..."


In [7]:
# Convert to list
df= df.values.tolist()

In [8]:
pprint(df[:1])

[['x']]


In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

df_words = list(sent_to_words(df))

print(df_words[:1])

[[]]


In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(df_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[df_words], threshold=100)  



/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [11]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)



In [12]:
# See trigram example
print(trigram_mod[bigram_mod[df_words[0]]])

[]


**Remove Stopwords, Make Bigrams and Lemmatize**

In [13]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [14]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [15]:
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [16]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Let’s call the functions in order.

In [26]:
# Remove Stop Words
df_words_nostops = remove_stopwords(df_words)

In [27]:
# Form Bigrams
df_words_bigrams = make_bigrams(df_words_nostops)

In [28]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])


In [30]:
# Do lemmatization keeping only noun, adj, vb, adv
df_lemmatized = lemmatization(df_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [31]:
print(df_lemmatized[:1])

[[]]


In [32]:
# Create Dictionary
id2word = corpora.Dictionary(df_lemmatized)

# Create Corpus
texts = df_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[]]


In [33]:
id2word[0]

'abolitionist'

or see this also

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abandon', 1),
  ('able', 1),
  ('abolition', 1),
  ('abolitionist', 1),
  ('absence', 3),
  ('absolute', 1),
  ('acceptable', 1),
  ('access', 1),
  ('accident', 1),
  ('accord', 9),
  ('account', 1),
  ('accurately', 1),
  ('achieve', 4),
  ('action', 4),
  ('activity', 3),
  ('actor', 2),
  ('actual', 1),
  ('additional', 1),
  ('administration', 2),
  ('adopt', 1),
  ('adrian', 1),
  ('advance', 1),
  ('advanced', 1),
  ('adversary', 1),
  ('advocate', 1),
  ('affair', 5),
  ('affect', 4),
  ('african', 1),
  ('afterlife', 1),
  ('age', 1),
  ('agent', 2),
  ('aggregate', 1),
  ('aggregation', 1),
  ('aggregative', 4),
  ('agonism', 1),
  ('agree', 3),
  ('aim', 1),
  ('allocate', 1),
  ('allow', 1),
  ('almost', 1),
  ('already', 1),
  ('also', 15),
  ('alter', 2),
  ('always', 1),
  ('american', 1),
  ('amount', 1),
  ('analysis', 1),
  ('analyze', 1),
  ('anarchism', 1),
  ('anarchist', 3),
  ('ancient', 2),
  ('anocracy', 1),
  ('antagonism', 1),
  ('anthony', 1),
  ('anthro

## LDA

In [37]:
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=id2word)

In [39]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(ldamodel, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [40]:
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.002382 -0.007918       1        1  43.939000
2     -0.021011 -0.003374       2        1  41.701461
1      0.021311 -0.002610       3        1  14.247086
3     -0.001822  0.015441       4        1   0.056253
4      0.003903 -0.001539       5        1   0.056201, topic_info=           Term       Freq      Total Category  logprob  loglift
82    political  89.000000  89.000000  Default  30.0000  30.0000
101       state  53.000000  53.000000  Default  29.0000  29.0000
81      politic  30.000000  30.000000  Default  28.0000  28.0000
98      society  23.000000  23.000000  Default  27.0000  27.0000
105      system  31.000000  31.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
68       nation   0.004536  11.088559   Topic5  -5.7425  -0.3175
51   individual   0.004135   9.653490   Topic5  -5.8352  -0.2717
76       people   0.004058   9.582552   Topic5  -5.8539  -0.2830
81      politic   0.005271  30.460256   Topic5  -5.5924  -1.1780
46        group   0.003815   8.660364   Topic5  -5.9156  -0.2436

[369 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         2  0.766604  abolitionist
907       1  0.390047       absence
907       3  0.390047       absence
122       1  0.546781        accord
122       2  0.410086        accord
...     ...       ...           ...
121       2  0.445121          work
325       1  0.351786         write
325       2  0.527679         write
326       1  0.761544     zeitgeist
326       2  0.761544     zeitgeist

[429 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 5])